# Calculating Primary Forest (Hansen) 

In [17]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

## Loading & Clipping Imagery

In [18]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/Muni')

# Import the Primary Tropical Forest cover image collection and filter it for Colombia
primaryForest_hansen = ee.ImageCollection("UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1").mosaic().selfMask()
primaryForest_hansen_COL = primaryForest_hansen.clip(colombiaMpios)

## Calculating Primary Forest Cover

In [19]:
# Calculate the loss area for the current year in square meters
def calculate_primary_forest_area(feature):

    primary_area_COL = primaryForest_hansen_COL.multiply(ee.Image.pixelArea())

    # Use reduceRegion to calculate the sum of the loss area within the feature's geometry
    primaryforest_area = primary_area_COL.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('Primary_HT_forests') 

    # Return a new feature with only the 'lossArea' and 'codmpio' properties
    return ee.Feature(None, {
        'primary_forest01': primaryforest_area,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

# Dictionary to store results for each year
results_dict = {}

# Use a lambda function to pass the current year into the map function
primaryforest_area01 = colombiaMpios.map(lambda feature: calculate_primary_forest_area(feature))
results_dict['primary_forest01'] = primaryforest_area01

# Optionally print the first feature's info to check
print(results_dict[f'primary_forest01'].first().getInfo())


{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'primary_forest01': 7743769.995038681}}


## Exporting to Google Drive

In [21]:
# Set description
description_file = 'Primary_Forest_2001'

# Export the FeatureCollection to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=results_dict['primary_forest01'],
    description=description_file,
    folder="EarthEngineExports",
    fileFormat='CSV'
)

# Start the export task
export_task.start()
print(f"Exporting {description_file} to Google Drive...")

Exporting Primary_Forest_2001 to Google Drive...
